<a href="https://colab.research.google.com/github/karrin10/Machine-Learning-for-the-Internet-of-Things/blob/Homework-2/hw2_template_pi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


> Kathleen Arrington

> HW2





In [ ]:
import tensorflow as tf
from tensorflow.keras import Input, layers, models, Model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
from PIL import Image

In [ ]:
# Define model1
def build_model1(input_shape=(32, 32, 3)):
    model1 = tf.keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3,3), strides=(2,2), activation="relu", padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, kernel_size=(3,3), strides=(2,2), activation="relu", padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, kernel_size=(3,3), strides=(2,2), activation="relu", padding='same'),
        layers.BatchNormalization()
    ])

    # Add four more pairs: Conv2D + BatchNorm
    for _ in range(4):
        model1.add(layers.Conv2D(128, kernel_size=(3,3), activation="relu", padding='same'))
        model1.add(layers.BatchNormalization())

    model1.add(layers.MaxPooling2D(pool_size=(4, 4)))
    model1.add(layers.Flatten())
    model1.add(layers.Dense(128, activation='relu'))
    model1.add(layers.BatchNormalization())
    model1.add(layers.Dense(10, activation='softmax'))

    return model1

In [ ]:
# Define model2
def build_model2(input_shape=(32, 32, 3)):
    model2 = tf.keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3,3), strides=(2,2), activation="relu", padding='same'),
        layers.BatchNormalization(),
        layers.SeparableConv2D(64, kernel_size=(3,3), strides=(2,2), activation="relu", padding='same'),
        layers.BatchNormalization(),
        layers.SeparableConv2D(128, kernel_size=(3,3), strides=(2,2), activation="relu", padding='same'),
        layers.BatchNormalization()
    ])

    # Add four more pairs: Conv2D + BatchNorm
    for _ in range(4):
        model2.add(layers.SeparableConv2D(128, kernel_size=(3,3), strides=(2,2), activation="relu", padding='same'))
        model2.add(layers.BatchNormalization())

    model2.add(layers.MaxPooling2D(pool_size=(1, 1)))
    model2.add(layers.Flatten())
    model2.add(layers.Dense(128, activation='relu'))
    model2.add(layers.BatchNormalization())
    model2.add(layers.Dense(10, activation='softmax'))

    return model2

In [ ]:
# Define model3
def build_model3(input_shape=(32, 32, 3)):
    # Apply input layer
    inputs = tf.keras.Input(shape=input_shape)

    # Incorporate convolutional block
    x = layers.Conv2D(32, kernel_size=(3,3), strides=(2,2), activation="relu", padding='same')(inputs)
    x = layers.BatchNormalization()(x)

    shortcut = x

    x = layers.Conv2D(64, kernel_size=(3,3), strides=(2,2), activation="relu", padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(128, kernel_size=(3,3), strides=(2,2), activation="relu", padding='same')(x)
    x = layers.BatchNormalization()(x)

    # After the fourth convolutional layer, skip connection
    shortcut = layers.Conv2D(128, kernel_size=(1, 1), strides=(4, 4), padding='same')(shortcut)
    x = layers.Add()([x, shortcut])
    shortcut2 = x
    # Add four more pairs: Conv2D + BatchNorm + Dropout
    for _ in range(2):
        x = layers.Conv2D(128, kernel_size=(3,3), activation="relu", padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
    x = layers.Add()([x, shortcut2])
    shortcut3 = x
    for _ in range(2):
        x = layers.Conv2D(128, kernel_size=(3,3), activation="relu", padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.2)(x)
    x = layers.Add()([x, shortcut3])


    x = layers.MaxPooling2D(pool_size=(4, 4))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(10, activation='softmax')(x)

    # Set model3
    model3 = tf.keras.Model(inputs=inputs, outputs=outputs)

    return model3

In [ ]:
# Define model50k
def build_model50k(input_shape=(32, 32, 3), dropout_rate=0.2):
    model50k = tf.keras.Sequential([
        layers.Input(shape=input_shape),
        layers.DepthwiseConv2D(kernel_size=(3,3), padding='same'),
        layers.Conv2D(32, kernel_size=(1,1), activation="relu", padding='same'),
        layers.BatchNormalization(),
        layers.Dropout(dropout_rate),
        layers.DepthwiseConv2D(kernel_size=(3,3), padding='same'),
        layers.Conv2D(64, kernel_size=(1,1), strides=(4,4), activation="relu", padding='same'),
        layers.BatchNormalization(),
        layers.Dropout(dropout_rate),
        layers.DepthwiseConv2D(kernel_size=(3,3), padding='same'),
        layers.Conv2D(96, kernel_size=(1,1), activation="relu", padding='same'),
        layers.BatchNormalization(),
        layers.Dropout(dropout_rate)
    ])
    model50k.add(layers.MaxPooling2D(pool_size=(4, 4)))
    model50k.add(layers.Flatten())
    model50k.add(layers.Dense(96, activation='relu'))
    model50k.add(layers.BatchNormalization())
    model50k.add(layers.Dropout(dropout_rate))
    model50k.add(layers.Dense(10, activation='softmax'))
    return model50k

In [ ]:
# no training or dataset construction should happen above this line
if __name__ == '__main__':

  ########################################
  (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
  class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
  train_labels = train_labels.squeeze()
  test_labels = test_labels.squeeze()

  input_shape = train_images.shape[1:]
  train_images = train_images / 255.0
  test_images = test_images / 255.0

  ########################################
  ## Build and train model 1
  model1 = build_model1()
  # compile and train model 1.
  # Train the model
  model1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  history = model1.fit(train_images, train_labels, epochs=50, validation_data=(test_images, test_labels))
  test_loss, test_acc = model1.evaluate(train_images, train_labels, verbose=2)

#image processing test for model 1
  image_path = "test_image_model1.png"
  image = Image.open(image_path)
  image = image.resize((32, 32))

  if image.mode != 'RGB':
    image = image.convert('RGB')

  # Convert image to numpy array and normalize
  image_array = np.array(image) / 255.0  # Normalize pixel values to [0, 1]

  # Expand dimensions to match the input shape expected by the model
  image_array = np.expand_dims(image_array, axis=0)

  # Make predictions
  predictions = model1.predict(image_array)

  # Get the predicted class
  predicted_class = np.argmax(predictions[0])

  # Print the predicted class
  print("Predicted class:", predicted_class)

#########################################
  ## Build, compile, and train model 2 (DS Convolutions)
  model2 = build_model2()
  # compile and train model 1.
  # Train the model
  model2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  history2 = model2.fit(train_images, train_labels, epochs=50, validation_data=(test_images, test_labels))
  test_loss, test_acc = model2.evaluate(train_images, train_labels, verbose=2)

#image processing test for model 2
  image_path = "test_image_model2.png"
  image = Image.open(image_path)
  image = image.resize((32, 32))

  if image.mode != 'RGB':
    image = image.convert('RGB')

  # Convert image to numpy array and normalize
  image_array = np.array(image) / 255.0  # Normalize pixel values to [0, 1]

  # Expand dimensions to match the input shape expected by the model
  image_array = np.expand_dims(image_array, axis=0)

  # Make predictions
  predictions = model2.predict(image_array)

  # Get the predicted class
  predicted_class = np.argmax(predictions[0])

  # Print the predicted class
  print("Predicted class:", predicted_class)

#########################################
  ### Repeat for model 3
  ## Build and train model 1
  model3 = build_model3()
  # compile and train model 1.
  # Train the model
  model3.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  history = model3.fit(train_images, train_labels, epochs=50, validation_data=(test_images, test_labels))
  test_loss, test_acc = model3.evaluate(train_images, train_labels, verbose=2)

#image processing test for model 3
  image_path = "test_image_model3.png"
  image = Image.open(image_path)
  image = image.resize((32, 32))

  if image.mode != 'RGB':
    image = image.convert('RGB')

  # Convert image to numpy array and normalize
  image_array = np.array(image) / 255.0  # Normalize pixel values to [0, 1]

  # Expand dimensions to match the input shape expected by the model
  image_array = np.expand_dims(image_array, axis=0)

  # Make predictions
  predictions = model1.predict(image_array)

  # Get the predicted class
  predicted_class = np.argmax(predictions[0])

  # Print the predicted class
  print("Predicted class:", predicted_class)

#########################################
  ### Repeat for model 50k
  ## Build and train model 50k
  model50k = build_model50k()
  # compile and train model 50k
  # Train the model
  model50k.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
  history = model50k.fit(train_images, train_labels, epochs=50, validation_data=(test_images, test_labels))
  test_loss, test_acc = model50k.evaluate(train_images, train_labels, verbose=2)
  # hw2_complete.py snippet for training and saving model50k
  model50k = build_model50k(input_shape=(32, 32, 3), dropout_rate=0.2)
  # Assume model50k is compiled and fitted here
  model50k.save("best_model.h5")


#image processing test for model 50k
  image_path = "Airplane.png"
  image = Image.open(image_path)
  image = image.resize((32, 32))

  if image.mode != 'RGB':
    image = image.convert('RGB')

  # Convert image to numpy array and normalize
  image_array = np.array(image) / 255.0  # Normalize pixel values to [0, 1]

  # Expand dimensions to match the input shape expected by the model
  image_array = np.expand_dims(image_array, axis=0)

  # Make predictions
  predictions = model1.predict(image_array)

  # Get the predicted class
  predicted_class = np.argmax(predictions[0])

  # Print the predicted class
  print("Predicted class:", predicted_class)

170498071/170498071 [==============================] - 2s 0us/step
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1563/1563 [==============================] - 172s 108ms/step - loss: 1.4752 - accuracy: 0.4656 - val_loss: 1.4797 - val_accuracy: 0.4779
Epoch 2/50
1563/1563 [==============================] - 167s 107ms/step - loss: 1.0936 - accuracy: 0.6130 - val_loss: 1.1574 - val_accuracy: 0.5959
Epoch 3/50
1563/1563 [==============================] - 171s 109ms/step - loss: 0.8973 - accuracy: 0.6854 - val_loss: 1.0855 - val_accuracy: 0.6284
Epoch 4/50
1563/1563 [==============================] - 167s 107ms/step - loss: 0.7395 - accuracy: 0.7413 - val_loss: 0.9950 - val_accuracy: 0.6639
Epoch 5/50
1563/1563 [==============================] - 167s 107ms/step - loss: 0.6149 - accuracy: 0.7856 - val_loss: 0.8347 - val_accuracy: 0.7192
Epoch 6/50
1563/1563 [==============================] - 167s 107ms/step - loss: 0.5079 - accuracy: 0.8223 - val_loss: 0.9777 - val_accuracy: 0.6858
Epoch 7/50
1563/1563 [==============================] - 169s 108ms/step - loss: 0.4094 - accuracy: 0.8570 - val_loss: 0.882

FileNotFoundError: [Errno 2] No such file or directory: 'test_image_model1.png'